In [ ]:
#%load_ext cudf


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

## **21_EEG**






In [ ]:
df_31_EEG = pd.read_csv('data/STData/31/31_EEG.csv')

In [ ]:
df_31_EEG.head()

In [ ]:
df_31_EEG.shape

In [ ]:
df_31_EEG.columns

In [ ]:
df_31_EEG.info()

In [ ]:
df_31_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_31_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_31_EEG['QuestionKey'].unique()

In [ ]:
df_31_EEG['TimeStamp'] = pd.to_datetime(df_31_EEG['TimeStamp'])

In [ ]:
df_31_EEG.describe()

In [ ]:
df_31_EEG.head(3)

In [ ]:
df_31_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_31_EEG['QuestionKey'].unique()

In [ ]:
df_31_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_31_EEG['QuestionKey'].value_counts()

In [ ]:
df_31_EEG['HeadBandOn'].unique()

In [ ]:
df_31_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_31_EEG.shape

In [ ]:
df_31_EEG.dropna(inplace=True)

In [ ]:
df_31_EEG.shape

In [ ]:
df_31_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_31_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_31_EEG.columns

In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
# from IPython.display import display, Markdown

# canvas = ds.Canvas(plot_width=1000, plot_height=600)

# for col in cols:
#     # Add a markdown cell before each plot for better separation and labeling
#     display(Markdown(f'### {col} over Time'))
#     agg = canvas.line(df_1_EEG, x='UnixTime', y=col)
#     img = tf.shade(agg)
#     plt.figure(figsize=(16, 10))
#     plt.imshow(img.to_pil())
#     # plt.axis('off')  # Removed this line to show axes
#     plt.xlabel("UnixTime") # Add x-axis label
#     plt.ylabel(col) # Add y-axis label
#     plt.show()

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_31_EEG['TimeStamp'], df_31_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_31_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_31_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_31_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_31_EEG[col].unique()}")

In [ ]:
df_31_EEG['HSI_TP9'].value_counts()

In [ ]:
df_31_EEG['HSI_TP10'].value_counts()

In [ ]:
df_31_EEG['HSI_AF7'].value_counts()

In [ ]:
df_31_EEG['HSI_AF8'].value_counts()

In [ ]:
df_31_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_31_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_31_EEG['HSI_TP9_flag'] = df_31_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_31_EEG['HSI_TP10_flag'] = df_31_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_31_EEG['HSI_AF7_flag'] = df_31_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_31_EEG['HSI_AF8_flag'] = df_31_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_31_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_31_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_31_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_31_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_31_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_31_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_31_EEG)
plt.show()

# **22 EGG**




In [ ]:
df_32_EEG = pd.read_csv('data/STData/32/32_EEG.csv')

In [ ]:
df_32_EEG.head()

In [ ]:
df_32_EEG.shape

In [ ]:
df_32_EEG.columns

In [ ]:
df_32_EEG.info()

In [ ]:
df_32_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_32_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_32_EEG['QuestionKey'].unique()

In [ ]:
df_32_EEG['TimeStamp'] = pd.to_datetime(df_32_EEG['TimeStamp'])

In [ ]:
df_32_EEG.describe()

In [ ]:
df_32_EEG.head(3)

In [ ]:
df_32_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_32_EEG['QuestionKey'].unique()

In [ ]:
df_32_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_32_EEG['QuestionKey'].value_counts()

In [ ]:
df_32_EEG['HeadBandOn'].unique()

In [ ]:
df_32_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_32_EEG.shape

In [ ]:
df_32_EEG.dropna(inplace=True)

In [ ]:
df_32_EEG.shape

In [ ]:
df_32_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_32_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_2_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_32_EEG['TimeStamp'], df_32_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_32_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_32_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_32_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_2_EEG[col].unique()}")

In [ ]:
df_32_EEG['HSI_TP9'].value_counts()

In [ ]:
df_32_EEG['HSI_TP10'].value_counts()

In [ ]:
df_32_EEG['HSI_AF7'].value_counts()

In [ ]:
df_32_EEG['HSI_AF8'].value_counts()

In [ ]:
df_32_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_32_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_32_EEG['HSI_TP9_flag'] = df_32_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_32_EEG['HSI_TP10_flag'] = df_32_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_32_EEG['HSI_AF7_flag'] = df_32_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_32_EEG['HSI_AF8_flag'] = df_32_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)2

In [ ]:
df_32_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_32_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_32_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_32_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_32_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_32_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_32_EEG)
plt.show()

**23 EGG**

In [ ]:
df_33_EEG = pd.read_csv('data/STData/33/33_EEG.csv')

In [ ]:
df_33_EEG.head()

In [ ]:
df_33_EEG.shape

In [ ]:
df_33_EEG.columns

In [ ]:
df_33_EEG.info()

In [ ]:
df_33_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_33_EEG.isnull(), cmap='YlGnBu')
plt.show()

In [ ]:
df_33_EEG['QuestionKey'].unique()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_33_EEG['QuestionKey'].unique()

In [ ]:
df_33_EEG['TimeStamp'] = pd.to_datetime(df_33_EEG['TimeStamp'])

In [ ]:
df_33_EEG.describe()

In [ ]:
df_33_EEG.head(3)

In [ ]:
df_33_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_33_EEG['QuestionKey'].unique()

In [ ]:
df_33_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_33_EEG['QuestionKey'].value_counts()

In [ ]:
df_33_EEG['HeadBandOn'].unique()

In [ ]:
df_33_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_33_EEG.shape

In [ ]:
df_33_EEG.dropna(inplace=True)

In [ ]:
df_33_EEG.shape

In [ ]:
df_33_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_33_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_33_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_33_EEG['TimeStamp'], df_33_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_33_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_33_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_33_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_33_EEG[col].unique()}")

In [ ]:
df_33_EEG['HSI_TP9'].value_counts()

In [ ]:
df_33_EEG['HSI_TP10'].value_counts()

In [ ]:
df_33_EEG['HSI_AF7'].value_counts()

In [ ]:
df_33_EEG['HSI_AF8'].value_counts()

In [ ]:
df_33_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_33_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_33_EEG['HSI_TP9_flag'] = df_33_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_33_EEG['HSI_TP10_flag'] = df_33_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_33_EEG['HSI_AF7_flag'] = df_33_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_33_EEG['HSI_AF8_flag'] = df_33_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_33_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_33_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_33_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_33_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_33_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_33_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_33_EEG)
plt.show()

# ***24 EGG***

In [ ]:
df_34_EEG = pd.read_csv('data/STData/34/34_EEG.csv')

In [ ]:
df_34_EEG.head()

In [ ]:
df_34_EEG.shape

In [ ]:
df_34_EEG.columns

In [ ]:
df_34_EEG.info()

In [ ]:
df_34_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_34_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_34_EEG['QuestionKey'].unique()

In [ ]:
df_34_EEG['TimeStamp'] = pd.to_datetime(df_4_EEG['TimeStamp'])

In [ ]:
df_34_EEG.describe()

In [ ]:
df_34_EEG.head(3)

In [ ]:
df_34_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_34_EEG['QuestionKey'].unique()

In [ ]:
df_34_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_34_EEG['QuestionKey'].value_counts()

In [ ]:
df_34_EEG['HeadBandOn'].unique()

In [ ]:
df_34_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_34_EEG.shape

In [ ]:
df_34_EEG.dropna(inplace=True)

In [ ]:
df_34_EEG.shape

In [ ]:
df_34_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_34_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_34_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_34_EEG['TimeStamp'], df_34_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_34_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_34_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_34_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_34_EEG[col].unique()}")

In [ ]:
df_34_EEG['HSI_TP9'].value_counts()

In [ ]:
df_34_EEG['HSI_TP10'].value_counts()

In [ ]:
df_34_EEG['HSI_AF7'].value_counts()

In [ ]:
df_34_EEG['HSI_AF8'].value_counts()

In [ ]:
df_34_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_34_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_34_EEG['HSI_TP9_flag'] = df_34_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_34_EEG['HSI_TP10_flag'] = df_34_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_34_EEG['HSI_AF7_flag'] = df_34_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_34_EEG['HSI_AF8_flag'] = df_34_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_34_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_34_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_34_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_34_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_34_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_34_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_34_EEG)
plt.show()

***25 EGG***

In [ ]:
df_35_EEG = pd.read_csv('data/STData/35/35_EEG.csv')

In [ ]:
df_35_EEG.head()

In [ ]:
df_35_EEG.shape

In [ ]:
df_35_EEG.columns

In [ ]:
df_35_EEG.info()

In [ ]:
df_35_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_35_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_35_EEG['QuestionKey'].unique()

In [ ]:
df_35_EEG['TimeStamp'] = pd.to_datetime(df_35_EEG['TimeStamp'])

In [ ]:
df_35_EEG.describe()

In [ ]:
df_35_EEG.head(3)

In [ ]:
df_35_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_35_EEG['QuestionKey'].unique()

In [ ]:
df_35_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_35_EEG['QuestionKey'].value_counts()

In [ ]:
df_35_EEG['HeadBandOn'].unique()

In [ ]:
df_35_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_35_EEG.shape

In [ ]:
df_35_EEG.dropna(inplace=True)

In [ ]:
df_35_EEG.shape

In [ ]:
df_35_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_35_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_35_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_35_EEG['TimeStamp'], df_35_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_35_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_35_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_35_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_35_EEG[col].unique()}")

In [ ]:
df_35_EEG['HSI_TP9'].value_counts()

In [ ]:
df_35_EEG['HSI_TP10'].value_counts()

In [ ]:
df_35_EEG['HSI_AF7'].value_counts()

In [ ]:
df_35_EEG['HSI_AF8'].value_counts()

In [ ]:
df_35_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_35_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_35_EEG['HSI_TP9_flag'] = df_35_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_35_EEG['HSI_TP10_flag'] = df_35_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_35_EEG['HSI_AF7_flag'] = df_35_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_35_EEG['HSI_AF8_flag'] = df_35_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_35_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_35_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_35_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_35_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_35_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_35_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_35_EEG)
plt.show()

# ***26 EGG***

In [ ]:
df_36_EEG = pd.read_csv('data/STData/36/36_EEG.csv')

In [ ]:
df_36_EEG.head()

In [ ]:
df_36_EEG.shape

In [ ]:
df_36_EEG.columns

In [ ]:
df_36_EEG.info()

In [ ]:
df_36_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_36_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_36_EEG['QuestionKey'].unique()

In [ ]:
df_36_EEG['TimeStamp'] = pd.to_datetime(df_6_EEG['TimeStamp'])

In [ ]:
df_36_EEG.describe()

In [ ]:
df_36_EEG.head(3)

In [ ]:
df_36_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_36_EEG['QuestionKey'].unique()

In [ ]:
df_36_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_36_EEG['QuestionKey'].value_counts()

In [ ]:
df_36_EEG['HeadBandOn'].unique()

In [ ]:
df_36_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_36_EEG.shape

In [ ]:
df_36_EEG.dropna(inplace=True)

In [ ]:
df_36_EEG.shape

In [ ]:
df_36_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_36_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_36_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_36_EEG['TimeStamp'], df_36_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_36_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_36_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_36_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_36_EEG[col].unique()}")

In [ ]:
df_36_EEG['HSI_TP9'].value_counts()

In [ ]:
df_36_EEG['HSI_TP10'].value_counts()

In [ ]:
df_36_EEG['HSI_AF7'].value_counts()

In [ ]:
df_36_EEG['HSI_AF8'].value_counts()

In [ ]:
df_36_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_36_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_36_EEG['HSI_TP9_flag'] = df_36_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_36_EEG['HSI_TP10_flag'] = df_36_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_36_EEG['HSI_AF7_flag'] = df_36_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_36_EEG['HSI_AF8_flag'] = df_36_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_36_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_36_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_36_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_36_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_36_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_36_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_36_EEG)
plt.show()

***27 EGG***

In [ ]:
df_37_EEG = pd.read_csv('data/STData/37/37_EEG.csv')

In [ ]:
df_37_EEG.head()

In [ ]:
df_37_EEG.shape

In [ ]:
df_37_EEG.columns

In [ ]:
df_37_EEG.info()

In [ ]:
df_37_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_37_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_37_EEG['QuestionKey'].unique()

In [ ]:
df_37_EEG['TimeStamp'] = pd.to_datetime(df_37_EEG['TimeStamp'])

In [ ]:
df_37_EEG.describe()

In [ ]:
df_37_EEG.head(3)

In [ ]:
df_37_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_37_EEG['QuestionKey'].unique()

In [ ]:
df_37_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_37_EEG['QuestionKey'].value_counts()

In [ ]:
df_37_EEG['HeadBandOn'].unique()

In [ ]:
df_37_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_37_EEG.shape

In [ ]:
df_37_EEG.dropna(inplace=True)

In [ ]:
df_37_EEG.shape

In [ ]:
df_37_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_37_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_37_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_37_EEG['TimeStamp'], df_37_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_37_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_37_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_37_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_37_EEG[col].unique()}")

In [ ]:
df_37_EEG['HSI_TP9'].value_counts()

In [ ]:
df_37_EEG['HSI_TP10'].value_counts()

In [ ]:
df_37_EEG['HSI_AF7'].value_counts()

In [ ]:
df_37_EEG['HSI_AF8'].value_counts()

In [ ]:
df_37_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_37_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_37_EEG['HSI_TP9_flag'] = df_37_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_37_EEG['HSI_TP10_flag'] = df_37_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_37_EEG['HSI_AF7_flag'] = df_37_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_37_EEG['HSI_AF8_flag'] = df_37_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_37_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_37_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_37_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_37_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_37_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_37_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_37_EEG)
plt.show()

***28 EGG***

In [ ]:
df_38_EEG.head()

In [ ]:
df_38_EEG = pd.read_csv('data/STData/8/8_EEG.csv')

In [ ]:
df_38_EEG.shape


In [ ]:
df_38_EEG.columns

In [ ]:
df_38_EEG.info()

In [ ]:
df_38_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_38_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_38_EEG['QuestionKey'].unique()

In [ ]:
df_38_EEG['TimeStamp'] = pd.to_datetime(df_38_EEG['TimeStamp'])

In [ ]:
df_38_EEG.describe()

In [ ]:
df_38_EEG.head(3)

In [ ]:
df_38_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_38_EEG['QuestionKey'].unique()

In [ ]:
df_38_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_38_EEG['QuestionKey'].value_counts()

In [ ]:
df_38_EEG['HeadBandOn'].unique()

In [ ]:
df_38_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_38_EEG.shape

In [ ]:
df_38_EEG.dropna(inplace=True)

In [ ]:
df_38_EEG.shape

In [ ]:
df_38_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_38_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_38_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_38_EEG['TimeStamp'], df_38_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_38_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_38_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_38_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_38_EEG[col].unique()}")

In [ ]:
df_38_EEG['HSI_TP9'].value_counts()

In [ ]:
df_38_EEG['HSI_TP10'].value_counts()

In [ ]:
df_38_EEG['HSI_AF7'].value_counts()

In [ ]:
df_38_EEG['HSI_AF8'].value_counts()

In [ ]:
df_38_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_38_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_38_EEG['HSI_TP9_flag'] = df_38_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_38_EEG['HSI_TP10_flag'] = df_38_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_38_EEG['HSI_AF7_flag'] = df_38_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_38_EEG['HSI_AF8_flag'] = df_38_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_38_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_38_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_38_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_38_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_38_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_38_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_38_EEG)
plt.show()

***29 EGG***

In [ ]:
df_29_EEG = pd.read_csv('data/STData/29/29_EEG.csv')

In [ ]:
df_29_EEG.head()

In [ ]:
df_29_EEG.shape

In [ ]:
df_29_EEG.columns

In [ ]:
df_29_EEG.info()

In [ ]:
df_29_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_29_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_29_EEG['QuestionKey'].unique()

In [ ]:
df_29_EEG['TimeStamp'] = pd.to_datetime(df_29_EEG['TimeStamp'])

In [ ]:
df_29_EEG.describe()

In [ ]:
df_29_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_29_EEG['QuestionKey'].unique()

In [ ]:
df_29_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_29_EEG['QuestionKey'].value_counts()

In [ ]:
df_29_EEG['HeadBandOn'].unique()

In [ ]:
df_29_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_29_EEG.shape

In [ ]:
df_29_EEG.dropna(inplace=True)

In [ ]:
df_29_EEG.shape

In [ ]:
df_29_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_29_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_29_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_29_EEG['TimeStamp'], df_29_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_29_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_29_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_29_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_29_EEG[col].unique()}")

In [ ]:
df_29_EEG['HSI_TP9'].value_counts()

In [ ]:
df_29_EEG['HSI_TP10'].value_counts()

In [ ]:
df_29_EEG['HSI_AF7'].value_counts()

In [ ]:
df_29_EEG['HSI_AF8'].value_counts()

In [ ]:
df_29_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_29_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_29_EEG['HSI_TP9_flag'] = df_29_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_29_EEG['HSI_TP10_flag'] = df_29_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_29_EEG['HSI_AF7_flag'] = df_29_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_29_EEG['HSI_AF8_flag'] = df_29_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_29_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_29_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_29_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_29_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_29_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_29_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_29_EEG)
plt.show()

***30 EGG***

In [ ]:
df_30_EEG = pd.read_csv('data/STData/30/30_EEG.csv')

In [ ]:
df_30_EEG.head()

In [ ]:
df_30_EEG.shape

In [ ]:
df_30_EEG.columns

In [ ]:
df_30_EEG.info()

In [ ]:
df_30_EEG.isnull().sum()

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_30_EEG.isnull(), cmap='YlGnBu')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` and `Elements` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  
- The `Elements` column is almost entirely null; dropping it likely won’t meaningfully impact our data quality.


In [ ]:
df_30_EEG['QuestionKey'].unique()

In [ ]:
df_30_EEG['TimeStamp'] = pd.to_datetime(df_30_EEG['TimeStamp'])

In [ ]:
df_30_EEG.describe()

In [ ]:
df_30_EEG.head(3)

In [ ]:
df_30_EEG.drop('Elements', axis=1, inplace=True)

In [ ]:
df_30_EEG['QuestionKey'].unique()

In [ ]:
df_30_EEG['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_30_EEG['QuestionKey'].value_counts()

In [ ]:
df_30_EEG['HeadBandOn'].unique()

In [ ]:
df_30_EEG['HeadBandOn'].isnull().sum()

In [ ]:
df_30_EEG.shape

In [ ]:
df_30_EEG.dropna(inplace=True)

In [ ]:
df_30_EEG.shape

In [ ]:
df_30_EEG.drop('HeadBandOn', axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df_30_EEG.isnull(), cmap='YlGnBu')
plt.show()

## Status Update

- All **null** (or missing) values in the dataset have been handled / eliminated.
- There are no remaining null values in any column.
- The dataset is now “complete” in the sense that every cell has a valid (non-null) entry.


In [ ]:
df_30_EEG.columns


In [ ]:
cols = ['Delta_TP9', 'Delta_AF7',
       'Delta_AF8', 'Delta_TP10', 'Theta_TP9', 'Theta_AF7', 'Theta_AF8',
       'Theta_TP10', 'Alpha_TP9', 'Alpha_AF7', 'Alpha_AF8', 'Alpha_TP10',
       'Beta_TP9', 'Beta_AF7', 'Beta_AF8', 'Beta_TP10', 'Gamma_TP9',
       'Gamma_AF7', 'Gamma_AF8', 'Gamma_TP10', 'RAW_TP9', 'RAW_AF7', 'RAW_AF8',
       'RAW_TP10', 'AUX_RIGHT', 'Accelerometer_X', 'Accelerometer_Y',
       'Accelerometer_Z', 'Gyro_X', 'Gyro_Y', 'Gyro_Z',
       'HSI_TP9', 'HSI_AF7', 'HSI_AF8', 'HSI_TP10', 'Battery']

In [ ]:
from IPython.display import display, Markdown


for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_30_EEG['TimeStamp'], df_30_EEG[col])
    plt.xlabel("TimeStamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_30_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)
plt.show()

## Notes & Observations

### Dropping `AUX_RIGHT`
- The `AUX_RIGHT` channel comes from the auxiliary electrode input of the EEG headset.  
- Upon inspection, its values appeared as **flat high-amplitude noise** (750–950 range) without any meaningful oscillatory EEG patterns.  
- Correlation analysis also showed **no significant relationship** between `AUX_RIGHT` and other EEG features.  
- Since it does not carry useful information and only adds noise/dimensionality, we **dropped `AUX_RIGHT`** from the dataset.




In [ ]:
df_30_EEG.drop('AUX_RIGHT', axis=1, inplace=True)

In [ ]:
df_30_EEG.head()

In [ ]:
HSI_cols = ["HSI_TP9", "HSI_AF7", "HSI_AF8", "HSI_TP10"]

In [ ]:
for col in HSI_cols:
    print(f"Unique values for {col}: {df_30_EEG[col].unique()}")

In [ ]:
df_30_EEG['HSI_TP9'].value_counts()

In [ ]:
df_30_EEG['HSI_TP10'].value_counts()

In [ ]:
df_30_EEG['HSI_AF7'].value_counts()

In [ ]:
df_30_EEG['HSI_AF8'].value_counts()

In [ ]:
df_30_EEG.describe()

## Notes & Observations

### Handling `HSI_*` Columns
- The `HSI_TP9`, `HSI_AF7`, `HSI_AF8`, `HSI_TP10` columns represent **Headset Signal Integrity** for each electrode:  
  - `1 = Good connection`  
  - `2 = Medium connection`  
  - `4 = Bad connection`  
- We need to pay attention to these values because:
  - Bad connections (`4`) indicate unreliable EEG readings.  
  - Medium connections (`2`) may still be usable but should be treated with caution.  
- Instead of dropping rows, we will **convert HSI values into binary flags**:  
  - **1 = Bad connection present**  
  - **0 = Otherwise (Good or Medium)**  
- This preserves all data while giving the model information about electrode reliability.  
- After creating these flags, the raw `HSI_*` columns can be removed to reduce dimensionality.


In [ ]:
df_30_EEG.head()

In [ ]:
signal_flags_values = { 1: 0, 2: 0, 4: 1}

In [ ]:
df_30_EEG['HSI_TP9_flag'] = df_30_EEG['HSI_TP9'].map(signal_flags_values).astype(np.int8)
df_30_EEG['HSI_TP10_flag'] = df_30_EEG['HSI_TP10'].map(signal_flags_values).astype(np.int8)
df_30_EEG['HSI_AF7_flag'] = df_30_EEG['HSI_AF7'].map(signal_flags_values).astype(np.int8)
df_30_EEG['HSI_AF8_flag'] = df_30_EEG['HSI_AF8'].map(signal_flags_values).astype(np.int8)

In [ ]:
df_30_EEG.drop(['HSI_TP9', 'HSI_TP10', 'HSI_AF7', 'HSI_AF8'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(38,12))
sns.heatmap(df_30_EEG.corr(numeric_only=True), cmap='YlGnBu', annot=True)a
plt.show()

## Notes & Observations

### Dropping RAW EEG Channels

- The EEG dataset includes `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` columns, which represent the unprocessed voltage readings from each electrode.  
- Upon inspection:
  - These RAW signals are **highly noisy** and show no clear oscillatory patterns typical of brainwave activity.  
  - Correlation analysis with other features shows **very low correlations** (near 0.0), indicating they do not contribute predictive value.  
  - Plots of RAW vs. time reveal large fluctuations without meaningful structure.

- Reason for dropping:
  - The dataset already provides **frequency band powers** (`Delta`, `Theta`, `Alpha`, `Beta`, `Gamma`) for each electrode, which are **derived from RAW signals** and are far more informative.  
  - Keeping RAW adds unnecessary dimensionality and noise, which can negatively affect machine learning models.

- Action taken:
  - `RAW_TP9`, `RAW_AF7`, `RAW_AF8`, `RAW_TP10` are **dropped from preprocessing**.  
  - The focus will be on the **precomputed band power features** for all EEG electrodes, which are sufficient for cognitive load prediction and regression tasks.


In [ ]:
df_30_EEG.drop(['RAW_TP9',	'RAW_AF7',	'RAW_AF8',	'RAW_TP10'], axis=1, inplace=True)

In [ ]:
df_30_EEG.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_30_EEG.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_30_EEG[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(38,12))
sns.pairplot(df_30_EEG)
plt.show()